# Geometry & Topoology

In this workshop we will learn about mesh data structures, how to import, visualize and export them in python. We will also learn about basic mesh manipulations.

## 0. Initialization

### 0.1. Importing libraries 

## 1. Mesh In

### 1.1. Importing OBJ files

### 1.2. Importing from Grasshopper (Speckle)

## 2. Mesh Manipulation & Analysis

### 2.1. Mesh Data Structure 

### 2.2. Basic Mesh Transformation

### 2.3. Dual

## 3. Mesh Out

### 3.1. Saving OBJ files

### 3.2. Exporting to Grasshopper (Speckle)

In [ ]:
__author__ = "Shervin Azadi"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/earthy_workshops"
__summary__ = "Earthy Design Studio Workshop on Geometry and Topology"